# Implementing Quantum Cryptography with the BB84 Protocol

In [ ]:
# Setup
from random import randint, sample

n = 50  # Length of the message to be sent

aliceBits = [randint(0, 1) for _ in range(n)]  # 0 or 1 for the random bits Alice generates
aliceBases = [randint(0, 1) for _ in range(n)]  # 0 = Z basis, 1 = X basis

In [ ]:
aliceBases

In [ ]:
# Encodes a message with the correct bits and bases
from qiskit import QuantumCircuit

def encodeMessage(bits, bases):
    message = []
    for bit, base in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if base == 0:  # Z basis
            if bit == 1:
                qc.x(0)  # Have to flip the qubit if the input bit is 1
        else:  # X basis
            if bit == 1:
                qc.x(0)  # Same as before for now
            qc.h(0)  # But now put it in superposition (recall: "rotation" from Z to X basis)
        qc.barrier()  # for readability
        message.append(qc)
    return message

In [ ]:
message = encodeMessage(aliceBits, aliceBases)  # create message as per above

# Look at some sample indices and their circuits
indices = sample(range(n), 5)
for i in indices:
    print(f"Bit: {aliceBits[i]}; Base: {aliceBases[i]}")
    print("——————————————————")

In [ ]:
message[indices[0]].draw(output="mpl");

In [ ]:
message[indices[1]].draw(output="mpl");

In [ ]:
message[indices[2]].draw(output="mpl");

In [ ]:
message[indices[3]].draw(output="mpl");

In [ ]:
message[indices[4]].draw(output="mpl");

In [ ]:
# Now on Bob's end: choose more random bases in which to measure the message

bobBases = [randint(0, 1) for _ in range(n)]

In [ ]:
# Measure the message using the given bases
from qiskit import Aer, assemble

def measureMessage(messageBits, bases):
    sim = Aer.get_backend("aer_simulator")
    meas = []
    for messageBit, base in zip(messageBits, bases):
        if base == 0:  # Z basis measurement
            messageBit.measure(0, 0)
        else:  # X basis measurement
            messageBit.h(0)  # "rotate" the basis again, since raw measurement can only be done in Z
            messageBit.measure(0, 0)
        qobj = assemble(messageBit, shots=1, memory=True)  # Only want one try, not the normal 1000, to mirror real-world situation
        # Run the circuit and fetch the measured bit from the classical register
        meas.append(int(sim.run(qobj).result().get_memory()[0]))
    return meas

In [ ]:
# Bob gets his results and compares bases with Alice

bobResults = measureMessage(message, bobBases)

# Let's look at the same 5 circuits from last time and see how they've changed.

In [ ]:
print(f"Message Bit: {aliceBits[indices[0]]}; Alice Base: {aliceBases[indices[0]]}; Bob Base: {bobBases[indices[0]]}")
message[indices[0]].draw(output="mpl");

In [ ]:
print(f"Message Bit: {aliceBits[indices[1]]}; Alice Base: {aliceBases[indices[1]]}; Bob Base: {bobBases[indices[1]]}")
message[indices[1]].draw(output="mpl");

In [ ]:
print(f"Message Bit: {aliceBits[indices[2]]}; Alice Base: {aliceBases[indices[2]]}; Bob Base: {bobBases[indices[2]]}")
message[indices[2]].draw(output="mpl");

In [ ]:
print(f"Message Bit: {aliceBits[indices[3]]}; Alice Base: {aliceBases[indices[3]]}; Bob Base: {bobBases[indices[3]]}")
message[indices[3]].draw(output="mpl");

In [ ]:
print(f"Message Bit: {aliceBits[indices[4]]}; Alice Base: {aliceBases[indices[4]]}; Bob Base: {bobBases[indices[4]]}")
message[indices[4]].draw(output="mpl");

In [ ]:
# Alice and Bob only keep the bits in which their bases were the same (so they are guaranteed to have made the same measurement).
def pruneInvalid(aBases, bBases, bits):
    validBits = []
    for aliceBase, bobBase, bit in zip(aBases, bBases, bits):
        if aliceBase == bobBase:  # bit is only valid if its bases were the same
            validBits.append(bit)
    return validBits

In [ ]:
from numpy import mod

# Alice and Bob will also publicly compare a subset of their final key to ensure that the protocol worked
def sampleBits(bits, sampleIndices):
    sampled = []
    for i in sampleIndices:
        # Have to calculate i modulo length of bits since we are changing the list length as we go, so we don't access indices out of range
        # We pop the element each time so it gets removed from the bits, since any bits they publicly share should not be part of their final secret key
        sampled.append(bits.pop(mod(i, len(bits))))
    return sampled

In [ ]:
# Both people generate keys using their own private bit string (Alice has the original bits, and Bob has his measured results)
# Remember that the bases are now publicly shared, so they both know each other's base choices.
aliceKey = pruneInvalid(aliceBases, bobBases, aliceBits)
bobKey = pruneInvalid(aliceBases, bobBases, bobResults)

sampleSize = 15
sampledBits = sample(range(n), sampleSize)  # using sample() prevents duplication

aliceSample = sampleBits(aliceKey, sampledBits)
bobSample = sampleBits(bobKey, sampledBits)

if aliceSample == bobSample:
    print("Protocol (most likely) worked!")

# The rest of their bits (those that were not shared) now form their secret key.

In this first demonstration, Alice and Bob have agreed on a secret classical key that they both know by using quantum information.
Notice that they transmitted classical information over an open communication channel, but this was never enough to deduce the key with certainty.
They took advantage of the randomness of quantum mechanics: as we will see, if someone were to intercept the transfer of quantum information (which actually holds the key itself), they would have a hard time getting away with it.

# Interception: Adding (and Catching) an Eavesdropper

In [ ]:
# First part of the protocol is the same: Alice randomly chooses bits and bases and encodes her message

aliceBits = [randint(0, 1) for _ in range(n)]
aliceBases = [randint(0, 1) for _ in range(n)]
message = encodeMessage(aliceBits, aliceBases)

In [ ]:
# Now Eve the malicious actor comes into the picture. Eve intercepts the qubits as they are being transferred from Alice to Bob for measurement (i.e. intercepts the quantum information).
# Eve chooses a random set of bases in which to measure the bits (this is the best she can do, as Alice and Bob have not shared their bases yet and will not do so until after Bob measures).

eveBases = [randint(0, 1) for _ in range(n)]
intercepted = measureMessage(message, eveBases)  # Eve measures the message, just like Bob will later, but in her own bases

In [ ]:
# Bob then does exactly what he would have done otherwise (because he doesn't know there was an eavesdropper)...
bobBases = [randint(0, 1) for _ in range(n)]
bobResults = measureMessage(message, bobBases)

# Let's look at five random bases again to see what has changed with Eve being a part of this exchange.
indices = sample(range(n), 5)

In [ ]:
print(f"Message Bit: {aliceBits[indices[0]]}; Alice Base: {aliceBases[indices[0]]}; Eve Base: {eveBases[indices[0]]}; Bob Base: {bobBases[indices[0]]}")
message[indices[0]].draw(output="mpl");

In [ ]:
print(f"Message Bit: {aliceBits[indices[1]]}; Alice Base: {aliceBases[indices[1]]}; Eve Base: {eveBases[indices[0]]}; Bob Base: {bobBases[indices[1]]}")
message[indices[1]].draw(output="mpl");

In [ ]:
print(f"Message Bit: {aliceBits[indices[2]]}; Alice Base: {aliceBases[indices[2]]}; Eve Base: {eveBases[indices[0]]}; Bob Base: {bobBases[indices[2]]}")
message[indices[2]].draw(output="mpl");

In [ ]:
print(f"Message Bit: {aliceBits[indices[3]]}; Alice Base: {aliceBases[indices[3]]}; Eve Base: {eveBases[indices[0]]}; Bob Base: {bobBases[indices[3]]}")
message[indices[3]].draw(output="mpl");

In [ ]:
print(f"Message Bit: {aliceBits[indices[4]]}; Alice Base: {aliceBases[indices[4]]}; Eve Base: {eveBases[indices[0]]}; Bob Base: {bobBases[indices[4]]}")
message[indices[4]].draw(output="mpl");

In [ ]:
# Now the key generation step happens as before: Alice and Bob reveal their bases and prune invalid bits

aliceKey = pruneInvalid(aliceBases, bobBases, aliceBits)
bobKey = pruneInvalid(aliceBases, bobBases, bobResults)

sampleSize = 15  # They again sample a subset of their qubits
sampledBits = sample(range(n), sampleSize)

aliceSample = sampleBits(aliceKey, sampledBits)
bobSample = sampleBits(bobKey, sampledBits)

print(aliceSample)
print(bobSample)

# In all likelihood, they are not equal! This is because Eve's measurements could have changed the quantum state by collapsing it, thereby skewing Bob's chances of measuring a certain outcome.
# Alice and Bob now know that there was an interception (or it could be due to noise in the communication channel, but let's disregard that for now).
# In this way, the protocol both enables secure key distribution and also has built-in detection that alerts the users when the channel might be compromised.

How likely is it that Eve goes undetected (that is, the interception is carried out but the bit strings still match)?
For any given bit, there is a 75% chance that Eve gets away with the interception: either she chooses the same basis as Alice, changing nothing (50%).
Or if not, she will change the state, but Bob still has a 50% chance of measuring the "right" qubit, and thinking nothing went wrong, so this adds another 50% of 50% (25%), for a total of 75%.
This might seem high, but since it's still less than 1, the chance that Eve gets away with this becomes vanishingly small with more sample bits.
With the 15 we used earlier, the chance is (0.75)^15 ≈ 0.0133, or 1.33% — and if we double it to 30 sample bits, the chance drops to below 0.01%!
This protocol is highly scalable and parallelizable, so adding a few bits to a key will not add much overhead in the large scheme.